In [3]:
from langchain_community.document_loaders import TextLoader, WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_ollama import ChatOllama
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.tools import Tool, tool, StructuredTool
from langchain_core.documents import Document
from langchain.agents import create_agent
from pydantic import BaseModel
from typing import List
from langgraph.graph import StateGraph, END

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
# Document Preprocessing

urls = [
    'https://lilianweng.github.io/posts/2023-06-23-agent/',
    'https://lilianweng.github.io/posts/2024-04-12-diffusion-video/'
]

loaders = [WebBaseLoader(url) for url in urls]

docs = []

for loader in loaders:
    docs.extend(loader.load())
    
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)

embedding = HuggingFaceEmbeddings(model="all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embedding)

retriever = vectorstore.as_retriever()

llm = ChatOllama(model="qwen3:4b")

In [ ]:
class RAGState(BaseModel):
    question: str
    sub_questions: List[str] = []
    retrieved_docs: List[Document] = []
    answer: str = ""

In [27]:
# Query Planner
def plan_query(state: RAGState) -> RAGState:
    
    prompt = f"""
        Break te foillowing complex question into 2-3 sub-questions:
        
        Question: {state.question}
        
        Sub-questions:
    """
    
    result = llm.invoke(prompt)
    sub_question = [line.strip("- ").strip() for line in result.content.strip().split("\n") if line.strip()]
    return RAGState(
        question=state.question,
        sub_questions=sub_question
    )
    
# Retrieve Document for each sub question
def retrieve_for_each(state: RAGState) -> RAGState:
    all_docs = []
    
    for sub in state.sub_questions:
        docs = retriever.invoke(sub)
        all_docs.extend(docs)
        
    return RAGState(
        question=state.question,
        sub_questions=state.sub_questions,
        retrieved_docs=all_docs
    )
    
# Generate final answer
def generate_final_answer(state: RAGState) -> RAGState:
    context = "\n\n".join([docs.page_content for docs in state.retrieved_docs])
    prompt = f""" 
        Use the context below to answer the question.
        
        Context:
        {context}
        
        Question:
        {state.question}        
    """
    
    answer = llm.invoke(prompt).content
    
    return RAGState(
        question=state.question,
        sub_questions=state.sub_questions,
        retrieved_docs=state.retrieved_docs,
        answer=answer
    )

In [28]:
# Generate Graph
builder = StateGraph(RAGState)

builder.add_node("planner", plan_query)
builder.add_node("retriever", retrieve_for_each)
builder.add_node("answer", generate_final_answer)

builder.set_entry_point("planner")
builder.add_edge("planner", "retriever")
builder.add_edge("retriever", "answer")
builder.add_edge("answer", END)

graph = builder.compile()

In [29]:
user_query = "Explain how agent loops work and what are the challenges in diffusion video generation?"
initial_state = RAGState(question=user_query)
final_state = graph.invoke(initial_state)
print(final_state)

print("\n Sub-question:")
for q in final_state['sub_questions']:
    print("-", q)
    
print("\n Final Answer:\n", final_state['answer'])


{'question': 'Explain how agent loops work and what are the challenges in diffusion video generation?', 'sub_questions': ['Here\'s a precise breakdown of your complex question into **3 targeted sub-questions** that maintain technical accuracy while resolving potential terminology confusion (note: "agent loops" is **not standard terminology** in diffusion video generation—this is addressed upfront to avoid misunderstanding):', '', '### 🔍 Sub-Questions:', '1. **Clarify terminology**: *What specific concept does "agent loops" refer to in the context of diffusion video generation?*', '*(Why? "Agent loops" isn\'t a standard term in diffusion models. This sub-question resolves confusion—e.g., it likely means "training loops," "temporal conditioning loops," or "agent-based control loops" in generative AI systems.)*', '2. **Core mechanism**: *How do diffusion models generate video sequences step-by-step (e.g., from latent space to frames)?*', '*(Why? This directly answers "how" for video diffu